In [3]:
import torch                           #dataloader 可用来从网上下载数据 
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#hyperparameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]    
)                                                           #转变数据，先变成tensor，再标准化（平均值和标准偏差）



In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform,download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)               #shuffle，数据的混合（对排序的数据应用）
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size)

100%|██████████| 170498071/170498071 [01:22<00:00, 2071579.91it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data


In [17]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')
'''
def imshow(img):
    img = img/2 +0.5 #unnormalize
    npimg= img.numpy()
    plt.show(np.transpose(npimg,(1,2,0)))
    plt.show()

#get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()


#show images
imshow(torchvision.utils.make_grid(images))

#for testing
conv1 = nn.Conv2d(3,6,5)            #3为个input的channel(颜色RGB), 6为输出的尺寸, 5为filter的大小kernel
pool = nn.MaxPool2d(2,2)            #数据的综合,用pool去使图片减少一半
conv2 = nn.Conv2d(6,16,5)           
print(images.shape)                 #4sample,3(RGB),32(大小),32(大小32*32)
x = conv1(images)
print(x.shape)                      #4,6,28,28, 32-5+1=28,详情间下一个code
x = pool(x)                         
print(x.shape)                      #4,6,14,14
x = conv2(x)                        #4,6,10,10
x = pool(x)                         ##4,6,5,5
'''

'\ndef imshow(img):\n    img = img/2 +0.5 #unnormalize\n    npimg= img.numpy()\n    plt.show(np.transpose(npimg,(1,2,0)))\n    plt.show()\n\n#get some random training images\ndataiter = iter(train_loader)\nimages, labels = dataiter.next()\n\n\n#show images\nimshow(torchvision.utils.make_grid(images))\n\n#for testing\nconv1 = nn.Conv2d(3,6,5)\npool = nn.MaxPool2d(2,2)\nconv2 = nn.Conv2d(6,16,5)\nprint(images.shape)\nx = conv1(images)\nprint(x.shape)\nx = pool(x)\nprint(x.shape)\nx = conv2(x)\nx = pool(x)\n'

In [20]:
#output_size : (W(像素点 of image,32*32) - F(5*5,filter) +2P（两边都有)/S(扫描间隔大小) +1 width,filtersize,padding,stride
#5*5 input, 3*3 f, p=0,stride = 1

In [28]:
#implement cover net, 有三组，每组颜色均需单独处理。 filter作用于input，依次进行扫描并更新；也可以用pool去综合矩阵信息，将2*2成为1，使矩阵变小
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)           #kernel_size 2 (2*2), stride = 2 (shift)，filter的扫描间隔;
        self.conv2 = nn.Conv2d(6,16,5)          #第二层卷积网络
        self.fc1 = nn.Linear(16*5*5, 120)       #卷积后接线性，120为隐藏层中神经元个数
        
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)                      #拉成一维进入线性网络
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
    
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [48]:
#start training
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 ==0:
            print(f'Epoch [{epoch+1}/{num_epochs}], step[{i+1}/{n_total_steps}], loss: {loss.item():.4f}')
            
print('Finished Training')

Epoch [1/4], step[2000/12500], loss: 0.7676
Epoch [1/4], step[4000/12500], loss: 1.0347
Epoch [1/4], step[6000/12500], loss: 0.2675
Epoch [1/4], step[8000/12500], loss: 1.7167
Epoch [1/4], step[10000/12500], loss: 0.9280
Epoch [1/4], step[12000/12500], loss: 0.2798
Epoch [2/4], step[2000/12500], loss: 2.1132
Epoch [2/4], step[4000/12500], loss: 0.4162
Epoch [2/4], step[6000/12500], loss: 1.0520
Epoch [2/4], step[8000/12500], loss: 0.6305
Epoch [2/4], step[10000/12500], loss: 1.0159
Epoch [2/4], step[12000/12500], loss: 0.8332
Epoch [3/4], step[2000/12500], loss: 1.7283
Epoch [3/4], step[4000/12500], loss: 0.2976
Epoch [3/4], step[6000/12500], loss: 0.9014
Epoch [3/4], step[8000/12500], loss: 1.4985
Epoch [3/4], step[10000/12500], loss: 0.5543
Epoch [3/4], step[12000/12500], loss: 1.0079
Epoch [4/4], step[2000/12500], loss: 1.9921
Epoch [4/4], step[4000/12500], loss: 1.7713
Epoch [4/4], step[6000/12500], loss: 0.7793
Epoch [4/4], step[8000/12500], loss: 1.1594
Epoch [4/4], step[10000/12

In [49]:
#Evaluation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] +=1
            n_class_samples[label] +=1
            
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}%')

Accuracy of the network: 61.35%
Accuracy of plane:64.1%
Accuracy of car:66.5%
Accuracy of bird:53.1%
Accuracy of cat:38.5%
Accuracy of deer:45.1%
Accuracy of dog:51.0%
Accuracy of frog:75.4%
Accuracy of horse:71.4%
Accuracy of ship:73.5%
Accuracy of truck:74.9%
